# Self Distillation rebuilt

## A new from the ground attempt at self distillation to try to sort out some of the issues that we faced in previous builds and testing.

## A new notebook for a new year.
---
## Primary issues
The main problems faced previously were to do with establishing a baseline of performance of a model to compare the self distilation work against. 


---


In [1]:
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import tensorflow as tf
os.environ['TF_DETERMINISTIC_OPS'] = '1'
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import os
import keras
sys.path.append("..") # Adds higher directory to python modules path.
import branchingdnn as branching
# dataset = branching.dataset.prepare.dataset(tf.keras.datasets.cifar10.load_data(),64,5000,22500,(227,227),include_targets=False)

In [2]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = tf.keras.models.clone_model(student)
    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=1,
    ):
        """ Configure the distiller.
        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data
        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                (teacher_predictions / self.temperature),
                (student_predictions / self.temperature),
            )
#             student_loss = student_loss #* self.alpha
#             distillation_loss = (distillation_loss) #* (1 - self.alpha)
            loss = distillation_loss
            #loss=student_loss +distillation_loss
        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"loss":loss,"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data
        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results

In [3]:
from tensorflow.keras import layers, models

def summarize_keras_trainable_variables(model, message):
    s = sum(map(lambda x: x.sum(), model.get_weights()))
    print("summary of trainable variables %s: %.13f" % (message, s))
    return s


In [4]:

# tf.debugging.experimental.enable_dump_debug_info(logdir, tensor_debug_mode="FULL_HEALTH", circular_buffer_size=-1)
(train_images, train_labels), (test_images, test_labels) = keras.datasets.cifar10.load_data()

CLASS_NAMES= ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# import csv
# with open('results/altTrain_labels.csv', newline='') as f:
    # reader = csv.reader(f,quoting=csv.QUOTE_NONNUMERIC)
    # alt_trainLabels = list(reader)
# with open('results/altTest_labels.csv', newline='') as f:
    # reader = csv.reader(f,quoting=csv.QUOTE_NONNUMERIC)
    # alt_testLabels = list(reader)

# altTraining = tf.data.Dataset.from_tensor_slices((train_images,alt_trainLabels))

# validation_images, validation_labels = train_images[:5000], alt_trainLabels[:5000]
# train_ds = tf.data.Dataset.from_tensor_slices((train_images, alt_trainLabels))
# test_ds = tf.data.Dataset.from_tensor_slices((test_images, alt_testLabels))
train_labels = tf.keras.utils.to_categorical(train_labels,10)
test_labels = tf.keras.utils.to_categorical(test_labels,10)

###normal method
validation_images, validation_labels = train_images[:5000], train_labels[:5000] #get the first 5k training samples as validation set
train_images, train_labels = train_images[5000:], train_labels[5000:] # now remove the validation set from the training set.
train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
validation_ds = tf.data.Dataset.from_tensor_slices((validation_images, validation_labels))

def augment_images(image, label):
    # Normalize images to have a mean of 0 and standard deviation of 1
    # image = tf.image.per_image_standardization(image)
    # Resize images from 32x32 to 277x277
    image = tf.image.resize(image, (227,227))
    return image, label

train_ds_size = len(list(train_ds))
test_ds_size = len(list(test_ds))
validation_ds_size = len(list(validation_ds))

train_ds = (train_ds
                  .map(augment_images)
                  .shuffle(buffer_size=train_ds_size,seed=42,reshuffle_each_iteration=False)
                  .batch(batch_size=32, drop_remainder=True))

test_ds = (test_ds
                  .map(augment_images)
                #   .shuffle(buffer_size=train_ds_size)
                  .batch(batch_size=32, drop_remainder=True))

validation_ds = (validation_ds
                  .map(augment_images)
                #   .shuffle(buffer_size=validation_ds_size)
                  .batch(batch_size=32, drop_remainder=True))


In [5]:
model_teacher = tf.keras.models.load_model("models/alexNetv6_logits_teacher.hdf5")

In [6]:
model_teacher.evaluate(test_ds)

312/312 [==============================] - 5s 12ms/step - loss: 0.6905 - accuracy: 0.7940


[0.6905280947685242, 0.7939703464508057]

## First, train the student model without the teacher input to get a baseline

In [10]:
seed = 66
# random.seed(seed)
tf.random.set_seed(seed)
np.random.seed(seed)
## keep setting the seed so that it doesn't matter what order you complete cells in. 
student_model = tf.keras.models.load_model("models/alexNetv6_logits_student.hdf5")
summarize_keras_trainable_variables(student_model,"before training")
student_model.fit(train_ds, validation_data = validation_ds, epochs=9)
summarize_keras_trainable_variables(student_model,"after training")


summary of trainable variables before training: 2688.9488805532455
Epoch 1/9
1406/1406 [==============================] - 97s 59ms/step - loss: 1.7786 - accuracy: 0.3969 - val_loss: 1.3743 - val_accuracy: 0.5226
Epoch 2/9
1406/1406 [==============================] - 84s 55ms/step - loss: 1.2942 - accuracy: 0.5343 - val_loss: 1.1749 - val_accuracy: 0.5883
Epoch 3/9
1406/1406 [==============================] - 84s 55ms/step - loss: 1.0954 - accuracy: 0.6092 - val_loss: 0.9928 - val_accuracy: 0.6462
Epoch 4/9
1406/1406 [==============================] - 84s 55ms/step - loss: 0.9501 - accuracy: 0.6607 - val_loss: 0.9316 - val_accuracy: 0.6707
Epoch 5/9
1406/1406 [==============================] - 85s 55ms/step - loss: 0.8374 - accuracy: 0.7029 - val_loss: 1.0647 - val_accuracy: 0.6146
Epoch 6/9
1406/1406 [==============================] - 90s 58ms/step - loss: 0.7421 - accuracy: 0.7394 - val_loss: 0.8288 - val_accuracy: 0.7071
Epoch 7/9
1406/1406 [==============================] - 86s 56ms

8335916.308644552

In [12]:
student_model.evaluate(test_ds)

312/312 [==============================] - 4s 12ms/step - loss: 0.7575 - accuracy: 0.7361


[0.7575096487998962, 0.7360777258872986]

In [7]:
seed = 66
# random.seed(seed)
tf.random.set_seed(seed)
np.random.seed(seed)
## keep setting the seed so that it doesn't matter what order you complete cells in. 
student_model = tf.keras.models.load_model("models/alexNetv6_logits_student.hdf5")
summarize_keras_trainable_variables(student_model,"before training")
student_model.fit(train_ds, validation_data = validation_ds, epochs=3)
summarize_keras_trainable_variables(student_model,"after training")

summary of trainable variables before training: 2688.9488805532455
Epoch 1/3
1406/1406 [==============================] - 92s 57ms/step - loss: 1.7787 - accuracy: 0.3966 - val_loss: 1.4048 - val_accuracy: 0.5000
Epoch 2/3
1406/1406 [==============================] - 82s 53ms/step - loss: 1.3243 - accuracy: 0.5234 - val_loss: 1.1543 - val_accuracy: 0.5998
Epoch 3/3
1406/1406 [==============================] - 82s 53ms/step - loss: 1.1354 - accuracy: 0.5913 - val_loss: 1.0041 - val_accuracy: 0.6478
summary of trainable variables after training: 9552084.5724323093891


9552084.57243231

In [8]:
student_model.evaluate(test_ds)

312/312 [==============================] - 4s 12ms/step - loss: 1.0213 - accuracy: 0.6436


[1.0212757587432861, 0.643629789352417]

In [8]:
seed = 66
# random.seed(seed)
tf.random.set_seed(seed)
np.random.seed(seed)
## keep setting the seed so that it doesn't matter what order you complete cells in. 
student_model = tf.keras.models.load_model("models/alexNetv6_logits_student.hdf5")
summarize_keras_trainable_variables(student_model,"before training")
student_model.fit(train_ds, validation_data = validation_ds, epochs=3)
summarize_keras_trainable_variables(student_model,"after training")


summary of trainable variables before training: 2688.9488805532455
Epoch 1/3
1406/1406 [==============================] - 101s 51ms/step - loss: 1.7787 - accuracy: 0.3966 - val_loss: 1.4048 - val_accuracy: 0.5000
Epoch 2/3
1406/1406 [==============================] - 81s 53ms/step - loss: 1.3243 - accuracy: 0.5234 - val_loss: 1.1543 - val_accuracy: 0.5998
Epoch 3/3
1406/1406 [==============================] - 103s 66ms/step - loss: 1.1354 - accuracy: 0.5913 - val_loss: 1.0041 - val_accuracy: 0.6478
summary of trainable variables after training: 9552084.5724323093891


9552084.57243231

In [9]:
student_model.evaluate(test_ds)

312/312 [==============================] - 4s 11ms/step - loss: 1.0213 - accuracy: 0.6436


[1.0212757587432861, 0.643629789352417]

## Next, train the student with the teacher model input as well to see the difference the teacher made. 

In [10]:
seed = 66
# random.seed(seed)
tf.random.set_seed(seed)
np.random.seed(seed)
def createDistiller(alpha=1):
    loaded_student = tf.keras.models.load_model("models/alexNetv6_logits_student.hdf5")
    summarize_keras_trainable_variables(loaded_student,"before compiling in distiller")
    distiller = Distiller(student=loaded_student, teacher=model_teacher)
    distiller.compile(
        optimizer=tf.optimizers.SGD(lr=0.001,momentum=0.9),
        metrics=[keras.metrics.CategoricalAccuracy()],
        student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=False),

        distillation_loss_fn=keras.losses.KLDivergence(),
        alpha=alpha,
        temperature=1,
    )
    return distiller


In [11]:
seed = 66
# random.seed(seed)
tf.random.set_seed(seed)
np.random.seed(seed)
distiller = createDistiller(1)
summarize_keras_trainable_variables(distiller.student,"before training")
# Distill teacher to student
distiller.fit(train_ds, validation_data = validation_ds,epochs=3,verbose=1)
# Evaluate student on test dataset
res = distiller.evaluate(test_ds)

summary of trainable variables before compiling in distiller: 2688.9488805532455


C:\Users\Sanity\AppData\Local\conda\conda\envs\py36\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


summary of trainable variables before training: 2684.4909174442291
Epoch 1/3
1406/1406 [==============================] - 117s 74ms/step - categorical_accuracy: 0.4011 - loss: 1.5589 - student_loss: 1.7577 - distillation_loss: 1.5589 - val_categorical_accuracy: 0.4545 - val_student_loss: 1.8797
Epoch 2/3
1406/1406 [==============================] - 121s 80ms/step - categorical_accuracy: 0.5406 - loss: 1.0852 - student_loss: 1.2874 - distillation_loss: 1.0852 - val_categorical_accuracy: 0.6056 - val_student_loss: 1.5514
Epoch 3/3
312/312 [==============================] - 4s 11ms/step - categorical_accuracy: 0.6655 - student_loss: 0.9662


In [9]:
seed = 66
# random.seed(seed)
tf.random.set_seed(seed)
np.random.seed(seed)
distiller = createDistiller(1)
summarize_keras_trainable_variables(distiller.student,"before training")
# Distill teacher to student
distiller.fit(train_ds, validation_data = validation_ds,epochs=3,verbose=1)
# Evaluate student on test dataset
res = distiller.evaluate(test_ds)

summary of trainable variables before compiling in distiller: 2688.9488805532455
summary of trainable variables before training: 2684.4909174442291
Epoch 1/3
1406/1406 [==============================] - 101s 64ms/step - categorical_accuracy: 0.4110 - loss: 1.5378 - student_loss: 1.7385 - distillation_loss: 1.5378 - val_categorical_accuracy: 0.4698 - val_student_loss: 1.8240
Epoch 2/3
1406/1406 [==============================] - 100s 64ms/step - categorical_accuracy: 0.5558 - loss: 1.0442 - student_loss: 1.2479 - distillation_loss: 1.0442 - val_categorical_accuracy: 0.5649 - val_student_loss: 1.5232
Epoch 3/3
312/312 [==============================] - 4s 12ms/step - categorical_accuracy: 0.6655 - student_loss: 0.9412


In [8]:
results = {}
results[0.1] = "test"
# for i in range(10):
seed = 66
# random.seed(seed)
tf.random.set_seed(seed)
np.random.seed(seed)
distiller = createDistiller(0.1)
summarize_keras_trainable_variables(distiller.student,"before training")
# Distill teacher to student
distiller.fit(train_ds, validation_data = validation_ds,epochs=9,verbose=1)
# Evaluate student on test dataset
res = distiller.evaluate(test_ds)
print("results: ",res)


summary of trainable variables before compiling in distiller: 2688.9488805532455


C:\Users\Sanity\AppData\Local\conda\conda\envs\py36\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


summary of trainable variables before training: 2684.4909174442291
Epoch 1/9
351/351 [==============================] - 124s 263ms/step - categorical_accuracy: 0.2968 - student_loss: 0.2443 - distillation_loss: 0.3866 - val_categorical_accuracy: 0.4323 - val_student_loss: 1.6946
Epoch 2/9
351/351 [==============================] - 89s 222ms/step - categorical_accuracy: 0.4244 - student_loss: 0.1675 - distillation_loss: 0.2981 - val_categorical_accuracy: 0.4914 - val_student_loss: 1.5332
Epoch 3/9
351/351 [==============================] - 96s 237ms/step - categorical_accuracy: 0.4865 - student_loss: 0.1473 - distillation_loss: 0.2653 - val_categorical_accuracy: 0.5407 - val_student_loss: 1.4427
Epoch 4/9
351/351 [==============================] - 89s 220ms/step - categorical_accuracy: 0.5312 - student_loss: 0.1350 - distillation_loss: 0.2419 - val_categorical_accuracy: 0.4559 - val_student_loss: 1.7602
Epoch 5/9
351/351 [==============================] - 80s 207ms/step - categorical_ac

NameError: name 'i' is not defined

In [15]:

tf.random.set_seed(seed)
np.random.seed(seed)
distiller = createDistiller(0.5)
summarize_keras_trainable_variables(distiller.student,"before training")
# Distill teacher to student
distiller.fit(train_ds, validation_data = validation_ds,epochs=9,verbose=1)
# Evaluate student on test dataset
res = distiller.evaluate(test_ds)
print("results: ",res)

tf.random.set_seed(seed)
np.random.seed(seed)
distiller = createDistiller(0.6)
summarize_keras_trainable_variables(distiller.student,"before training")
# Distill teacher to student
distiller.fit(train_ds, validation_data = validation_ds,epochs=9,verbose=1)
# Evaluate student on test dataset
res = distiller.evaluate(test_ds)
print("results: ",res)

tf.random.set_seed(seed)
np.random.seed(seed)
distiller = createDistiller(0.7)
summarize_keras_trainable_variables(distiller.student,"before training")
# Distill teacher to student
distiller.fit(train_ds, validation_data = validation_ds,epochs=9,verbose=1)
# Evaluate student on test dataset
res = distiller.evaluate(test_ds)
print("results: ",res)

tf.random.set_seed(seed)
np.random.seed(seed)
distiller = createDistiller(0.8)
summarize_keras_trainable_variables(distiller.student,"before training")
# Distill teacher to student
distiller.fit(train_ds, validation_data = validation_ds,epochs=9,verbose=1)
# Evaluate student on test dataset
res = distiller.evaluate(test_ds)
print("results: ",res)

tf.random.set_seed(seed)
np.random.seed(seed)
distiller = createDistiller(1)
summarize_keras_trainable_variables(distiller.student,"before training")
# Distill teacher to student
distiller.fit(train_ds, validation_data = validation_ds,epochs=9,verbose=1)
# Evaluate student on test dataset
res = distiller.evaluate(test_ds)
print("results: ",res)
    

summary of trainable variables before compiling in distiller: 2688.9488805532455


C:\Users\Sanity\AppData\Local\conda\conda\envs\py36\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


summary of trainable variables before training: 2684.4909174442291
Epoch 1/9
1406/1406 [==============================] - 103s 65ms/step - categorical_accuracy: 0.3953 - student_loss: 0.8973 - distillation_loss: 0.1748 - val_categorical_accuracy: 0.4946 - val_student_loss: 1.7729
Epoch 2/9
1406/1406 [==============================] - 101s 65ms/step - categorical_accuracy: 0.5283 - student_loss: 0.6598 - distillation_loss: 0.1342 - val_categorical_accuracy: 0.5897 - val_student_loss: 1.5725
Epoch 3/9
1406/1406 [==============================] - 109s 69ms/step - categorical_accuracy: 0.5977 - student_loss: 0.5637 - distillation_loss: 0.1136 - val_categorical_accuracy: 0.6210 - val_student_loss: 1.6123
Epoch 4/9
1406/1406 [==============================] - 100s 65ms/step - categorical_accuracy: 0.6509 - student_loss: 0.4930 - distillation_loss: 0.0983 - val_categorical_accuracy: 0.5873 - val_student_loss: 1.7226
Epoch 5/9
1406/1406 [==============================] - 104s 69ms/step - categ

KeyboardInterrupt: 

In [19]:
for i in range(3):
    seed = 66
    # random.seed(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)
    distiller = createDistiller(0.1)
#     summarize_keras_trainable_variables(distiller.student,"before training")
    # Distill teacher to student
    tf.random.set_seed(seed)
    np.random.seed(seed)
    model = tf.keras.models.load_model("models/alexNetv6_logits_student.hdf5")
    model.compile(loss='categorical_crossentropy', optimizer=tf.optimizers.SGD(lr=0.001,momentum=0.9), metrics=['accuracy'])
    model.fit(train_ds, validation_data=validation_ds, epochs=1,verbose=1)
    model.evaluate(test_ds)
#     distiller.fit(train_ds, validation_data = validation_ds,epochs=1,verbose=1)
    # Evaluate student on test dataset
#     res = distiller.evaluate(test_ds)
#     print("results: ",res)


summary of trainable variables before compiling in distiller: 2688.9488805532455
312/312 [==============================] - 4s 13ms/step - loss: 1.3680 - accuracy: 0.5146
summary of trainable variables before compiling in distiller: 2688.9488805532455
312/312 [==============================] - 4s 13ms/step - loss: 1.3786 - accuracy: 0.5082
summary of trainable variables before compiling in distiller: 2688.9488805532455
312/312 [==============================] - 4s 13ms/step - loss: 1.4670 - accuracy: 0.4779


In [21]:

# tf.debugging.experimental.enable_dump_debug_info(logdir, tensor_debug_mode="FULL_HEALTH", circular_buffer_size=-1)
(train_images, train_labels), (test_images, test_labels) = keras.datasets.cifar10.load_data()

CLASS_NAMES= ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# import csv
# with open('results/altTrain_labels.csv', newline='') as f:
    # reader = csv.reader(f,quoting=csv.QUOTE_NONNUMERIC)
    # alt_trainLabels = list(reader)
# with open('results/altTest_labels.csv', newline='') as f:
    # reader = csv.reader(f,quoting=csv.QUOTE_NONNUMERIC)
    # alt_testLabels = list(reader)

# altTraining = tf.data.Dataset.from_tensor_slices((train_images,alt_trainLabels))

# validation_images, validation_labels = train_images[:5000], alt_trainLabels[:5000]
# train_ds = tf.data.Dataset.from_tensor_slices((train_images, alt_trainLabels))
# test_ds = tf.data.Dataset.from_tensor_slices((test_images, alt_testLabels))
train_labels = tf.keras.utils.to_categorical(train_labels,10)
test_labels = tf.keras.utils.to_categorical(test_labels,10)

###normal method
validation_images, validation_labels = train_images[:5000], train_labels[:5000] #get the first 5k training samples as validation set
train_images, train_labels = train_images[5000:], train_labels[5000:] # now remove the validation set from the training set.
train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
validation_ds = tf.data.Dataset.from_tensor_slices((validation_images, validation_labels))

def augment_images(image, label):
    # Normalize images to have a mean of 0 and standard deviation of 1
    # image = tf.image.per_image_standardization(image)
    # Resize images from 32x32 to 277x277
    image = tf.image.resize(image, (227,227))
    return image, label

train_ds_size = len(list(train_ds))
test_ds_size = len(list(test_ds))
validation_ds_size = len(list(validation_ds))

train_ds = (train_ds
                  .map(augment_images)
                  .shuffle(buffer_size=train_ds_size,seed=42,reshuffle_each_iteration=False)
                  .batch(batch_size=128, drop_remainder=True))

test_ds = (test_ds
                  .map(augment_images)
                #   .shuffle(buffer_size=train_ds_size)
                  .batch(batch_size=128, drop_remainder=True))

validation_ds = (validation_ds
                  .map(augment_images)
                #   .shuffle(buffer_size=validation_ds_size)
                  .batch(batch_size=128, drop_remainder=True))

for i in range(3):
    tf.random.set_seed(seed)
    np.random.seed(seed)
    model = tf.keras.models.load_model("distil_test.hdf5")
    summarize_keras_trainable_variables(model,"before training")
    
    model.fit(train_ds, shuffle=False,validation_data = validation_ds,epochs=1)
    summarize_keras_trainable_variables(model,"after training")
    results = model.evaluate(test_ds)
    print(results)

summary of trainable variables before training: 2931.7305606603622
351/351 [==============================] - 180s 314ms/step - loss: 1.9348 - accuracy: 0.3622 - val_loss: 1.5802 - val_accuracy: 0.4387
summary of trainable variables after training: 2272256.2266048272140
78/78 [==============================] - 3s 42ms/step - loss: 1.5863 - accuracy: 0.4361
[1.5863159894943237, 0.43609777092933655]
summary of trainable variables before training: 2931.7305606603622
351/351 [==============================] - 80s 195ms/step - loss: 1.9348 - accuracy: 0.3622 - val_loss: 1.5802 - val_accuracy: 0.4387
summary of trainable variables after training: 2272256.2266048272140
78/78 [==============================] - 3s 43ms/step - loss: 1.5863 - accuracy: 0.4361
[1.5863159894943237, 0.43609777092933655]
summary of trainable variables before training: 2931.7305606603622
351/351 [==============================] - 81s 203ms/step - loss: 1.9348 - accuracy: 0.3622 - val_loss: 1.5802 - val_accuracy: 0.438

In [14]:
for i in range(10):
    print(i/100)


0.0
0.01
0.02
0.03
0.04
0.05
0.06
0.07
0.08
0.09


In [ ]:
seed = 42
# random.seed(seed)
tf.random.set_seed(seed)
np.random.seed(seed)
## keep setting the seed so that it doesn't matter what order you complete cells in. 
student_model = tf.keras.models.load_model("models/alexNetv6_logits_student.hdf5")
summarize_keras_trainable_variables(student_model,"before training")
student_model.fit(train_ds, validation_data = validation_ds, epochs=3)
summarize_keras_trainable_variables(student_model,"after training")

In [15]:
outputs =[]
inputs = keras.Input(shape=(28,28,))
x = layers.Flatten(input_shape=(28,28))(inputs)
x = layers.Dense(512, activation="relu")(x)
#exit 1 The main branch exit is refered to as "exit 1" or "main exit" to avoid confusion when adding addtional exits
output1 = layers.Dense(10, name="output1")(x)
softmax = layers.Softmax()(output1)

outputs.append(softmax)
print(len(outputs))
model_student = keras.Model(inputs=inputs, outputs=outputs, name="mnist_model_normal")
model_student.summary()
#visualize_model(model,"mnist_normal")
print(len(model_student.outputs))


1
Model: "mnist_model_normal"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 28, 28)]          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               401920    
_________________________________________________________________
output1 (Dense)              (None, 10)                5130      
_________________________________________________________________
softmax (Softmax)            (None, 10)                0         
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________
1


In [ ]:
inputs = keras.Input(shape=(227,227,3))
# targets = keras.Input(shape=(10,))
x = keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(227,227,3))(inputs)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2))(x)
x = keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same")(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2))(x)
x = keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same")(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Conv2D(filters=384, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same")(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same")(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(4096, activation='relu')(x)
x = keras.layers.Dropout(0.5)(x)

# ### first branch
# branchLayer = keras.layers.Flatten(name=tf.compat.v1.get_default_graph().unique_name("branch_flatten"))(x)
# branchLayer = keras.layers.Dense(124, activation="relu",name=tf.compat.v1.get_default_graph().unique_name("branch124"))(branchLayer)
# branchLayer = keras.layers.Dense(64, activation="relu",name=tf.compat.v1.get_default_graph().unique_name("branch64"))(branchLayer)
# branchLayer = keras.layers.Dense(10, name=tf.compat.v1.get_default_graph().unique_name("branch_output"))(branchLayer)

x = keras.layers.Dense(4096, activation='relu')(x)
x = keras.layers.Dropout(0.5)(x)

# ### second Branch
# branchLayer2 = keras.layers.Flatten(name=tf.compat.v1.get_default_graph().unique_name("branch_flatten"))(x)
# branchLayer2 = keras.layers.Dense(10, name=tf.compat.v1.get_default_graph().unique_name("branch_output"))(branchLayer2)

x = keras.layers.Dense(10, activation='softmax')(x)

# model = keras.Model(inputs=inputs, outputs=[x,branchLayer,branchLayer2], name="alexnet")
teacher = keras.Model(inputs=(inputs), outputs=[x], name="alexnet")

teacher.compile(loss='categorical_crossentropy', optimizer=tf.optimizers.SGD(lr=0.001,momentum=0.9), metrics=['accuracy'])
teacher.save("models/alexNetv6_new_teacher.hdf5")

In [ ]:
summarize_keras_trainable_variables(student_model,"before training")
teacher.fit(train_ds, validation_data = validation_ds, epochs=30)
summarize_keras_trainable_variables(student_model,"after training")
teacher.save("models/alexNetv6_new_teacher.hdf5")
